In [1]:
from collections import defaultdict
import math
import time
import random
import os, sys

import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

In [2]:

N = 2 # The length of the n-gram
EMB_SIZE = 128 # The size of the embedding

embeddings_location = "embeddings.txt" #the file to write the word embeddings to
labels_location = "labels.txt" #the file to write the labels to

USE_CUDA = torch.cuda.is_available()

# Functions to read in the corpus
# NOTE: We are using data from the Penn Treebank, which is already converted
#       into an easy-to-use format with "<unk>" symbols. If we were using other
#       data we would have to do pre-processing and consider how to choose
#       unknown words, etc.
w2i = defaultdict(lambda: len(w2i))
S = w2i["<s>"]
UNK = w2i["<unk>"]
def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            yield [w2i[x] for x in line.strip().split(" ")]

# Read in the data
train = list(read_dataset("../data/ptb/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/ptb/valid.txt"))
i2w = {v: k for k, v in w2i.items()}
nwords = len(w2i)
nbits = len(np.binary_repr(nwords-1))

with open(labels_location, 'w') as labels_file:
    for i in range(nwords):
        labels_file.write(i2w[i] + '\n')


In [3]:

class SKIP_BINARY(nn.Module):
    def __init__(self, nwords, emb_size,nbits):
        super(SKIP_BINARY, self).__init__()
        self.embedding = nn.Embedding(nwords, emb_size)
        self.linear = nn.Linear(emb_size, nbits)

    def forward(self, words):
        #print(words)
        emb = self.embedding(words)    # 2D Tensor of size [batch_size x emb_size]
        #feat = emb.view(emb.size(0), -1) # 2D Tensor of size [batch_size x (2 x num_context*emb_size)]
        logit = self.linear(emb)           # 2D Tensor of size [batch_size x nbits]
        #print(logit)
        return logit

In [4]:
# Initialize the model and the optimizer
model = SKIP_BINARY(nwords=nwords, emb_size=EMB_SIZE,nbits=nbits)
if USE_CUDA:
    model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
# convert a (nested) list of int into a pytorch Variable
def convert_to_variable(words):
    var = Variable(torch.LongTensor(words))
    if USE_CUDA:
        var = var.cuda()

    return var

# A function to calculate scores for one value
def calc_score_of_histories(words):
    # This will change from a list of histories, to a pytorch Variable whose data type is LongTensor
    #print(words)
    words_var = convert_to_variable(words)
    logits = model(words_var)
    return logits

# Calculate the loss value for the entire sentence
def calc_sent_loss(sent):
    # The initial history is equal to end of sentence symbols
    # Step through the sentence, including the end of sentence token
    all_histories = []
    all_targets = []
    for i in range(0,len(sent)):
        pos_words = ([sent[x] if x >= 0 else S for x in range(i-N,i)] +
                 [sent[x] if x < len(sent) else S for x in range(i+1,i+N+1)])
        word_repr = [[float(y) for y in np.binary_repr(x).zfill(nbits)] for x in pos_words]
        all_histories.extend(pos_words)
        all_targets.extend(word_repr)
    all_targets=Variable(torch.FloatTensor(all_targets))
    if USE_CUDA:
        all_targets = all_targets.cuda()    
    predictions=calc_score_of_histories(all_histories)
    loss=nn.functional.binary_cross_entropy_with_logits(predictions, all_targets, size_average=False)
    return loss

In [6]:
last_dev = 1e20
best_dev = 1e20

for ITER in range(2):
  # Perform training
    random.shuffle(train)
    # set the model to training mode
    model.train()
    train_words, train_loss = 0, 0.0
    start = time.time()
    for sent_id, sent in enumerate(train):
        if len(sent)<=1:
            continue
        #print(sent)
        my_loss = calc_sent_loss(sent)
        train_loss += my_loss.data[0]
        train_words += len(sent)
        optimizer.zero_grad()
        my_loss.backward()
        optimizer.step()
        if (sent_id+1) % 5000 == 0:
            print("--finished %r sentences (word/sec=%.2f)" % (sent_id+1, train_words/(time.time()-start)))
    print("iter %r: train loss/word=%.4f, ppl=%.4f (word/sec=%.2f)" % (ITER, train_loss/train_words, math.exp(train_loss/train_words), train_words/(time.time()-start)))

    # Evaluate on dev set
    # set the model to evaluation mode
    model.eval()
    dev_words, dev_loss = 0, 0.0
    start = time.time()
    for sent_id, sent in enumerate(dev):
        if len(sent)<=1:
            continue
        my_loss = calc_sent_loss(sent)
        dev_loss += my_loss.data[0]
        dev_words += len(sent)

    # Keep track of the development accuracy and reduce the learning rate if it got worse
    if last_dev < dev_loss:
        optimizer.learning_rate /= 2
    last_dev = dev_loss

    # Keep track of the best development accuracy, and save the model only if it's the best one
    if best_dev > dev_loss:
        print("saving embedding files")
        with open(embeddings_location, 'w') as embeddings_file:
            W_w_np = model.embedding.weight.cpu().data.numpy()
            for i in range(nwords):
                ith_embedding = '\t'.join(map(str, W_w_np[i]))
                embeddings_file.write(ith_embedding + '\n')
        best_dev = dev_loss
        
        # Save the model
    print("iter %r: dev loss/word=%.4f, ppl=%.4f (word/sec=%.2f)" % (ITER, dev_loss/dev_words, math.exp(dev_loss/dev_words), dev_words/(time.time()-start)))


--finished 5000 sentences (word/sec=5241.69)
--finished 10000 sentences (word/sec=5213.65)
--finished 15000 sentences (word/sec=5208.77)
--finished 20000 sentences (word/sec=5184.58)
--finished 25000 sentences (word/sec=5267.74)
--finished 30000 sentences (word/sec=5381.84)
--finished 35000 sentences (word/sec=5462.38)
--finished 40000 sentences (word/sec=5551.72)
iter 0: train loss/word=5.5509, ppl=257.4751 (word/sec=5580.24)
saving embedding files
iter 0: dev loss/word=1.8138, ppl=6.1335 (word/sec=11344.42)
--finished 5000 sentences (word/sec=6096.37)
--finished 10000 sentences (word/sec=6206.39)
--finished 15000 sentences (word/sec=6199.85)
--finished 20000 sentences (word/sec=6234.28)
--finished 25000 sentences (word/sec=6234.60)
--finished 30000 sentences (word/sec=6225.82)
--finished 35000 sentences (word/sec=6213.15)
--finished 40000 sentences (word/sec=6219.90)
iter 1: train loss/word=0.6473, ppl=1.9103 (word/sec=6219.88)
saving embedding files
iter 1: dev loss/word=0.6981, ppl